In [2]:
from pdf2image import convert_from_path
import os

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    images = convert_from_path(pdf)
    for i in range(len(images)):
        images[i].save('hotel_stay_images\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')

path ="Hotel Expenses"

#store all the file names in this list
filelist = []

for root, dirs, files in os.walk(path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))


# print all the file names
for file in filelist:
    filename = os.path.split(file)[-1].split('.pdf')[0]
    if ' ' in filename:
        filename = filename.replace(' ', '_')
    # print(filename)
    pdf_to_images(file, filename)


In [ ]:
from prettyprinter import pprint
import glob
import datetime
import pandas as pd

def get_traveler_name(text):
    traveler_name = ''
    for i in range(len(text)):
        if 'mr' in text[i].lower():
            if 'forma' in text[i].lower():
                words = text[i].split()
                newwords = [word.lower() for word in words]
                for i in range(len(newwords)):
                    if 'forma' in newwords[i]:
                        ind = i
                        break
                traveler_name = ' '.join(words[:ind])
            else:
                traveler_name = text[i]
            return traveler_name

def get_dates(text):
    start_date = ''
    end_date = ''
    for i in range(len(text)):
        if 'arrival' in text[i].lower():
            start_date = text[i].split()[-1]
            start_date = start_date.replace('‘', '')
            start_date = datetime.datetime.strptime(start_date, '%d.%m.%y').strftime('%Y-%m-%d')
            end_date = text[i+1].split()[-1]
            end_date = end_date.replace('‘', '')
            end_date = datetime.datetime.strptime(end_date, '%d.%m.%y').strftime('%Y-%m-%d')
            return start_date, end_date

def get_number_of_rooms(text):
    no_of_rooms = 0
    for line in text:
        if 'room :' in line.lower():
            words = line.split()
            if len(words) > 2:
                no_of_rooms = len(words[2:])
                return no_of_rooms
            else:
                return 1

def get_number_of_nights(text):
    no_of_nights = 0
    try:
        start_date = get_dates(text)[0]
        fyear, fmonth, fday = start_date.split('-')
        fdate = datetime.date(int(fyear), int(fmonth), int(fday))
        end_date = get_dates(text)[1]
        lyear, lmonth, lday = end_date.split('-')
        ldate = datetime.date(int(lyear), int(lmonth), int(lday))
        return (ldate-fdate).days
    except:
        pass

def get_hotel_name(text):
    hotel_name = ''
    for line in text:
        if 'llc' in line.lower():
            if 'india' in line.lower():
                words = line.split()
                newwords = [word.lower() for word in words]
                for i in range(len(newwords)):
                    if 'india' in newwords[i]:
                        ind = i
                        break
                hotel_name = ' '.join(words[ind+1:])
            elif 'swarnabhoomi' in line.lower():
                words = line.split()
                newwords = [word.lower() for word in words]
                for i in range(len(newwords)):
                    if 'swarnabhoomi' in newwords[i]:
                        ind = i
                        break
                hotel_name = ' '.join(words[ind+1:])
            else:
                hotel_name = line
            return hotel_name

def get_hotel_city(text):
    hotel_city = ''
    for line in text:
        if 'tel' in line.lower() and 'box' in line.lower():
            words = line.split()
            newwords = [word.lower() for word in words]
            for i in range(len(newwords)):
                if 'tel' in newwords[i]:
                    ind = i
                    break
            hotel_city = words[ind-1]
            return hotel_city.strip(',')

def get_hotel_country(text, data):
    hotel_country = ''
    hotel_city = get_hotel_city(text)
    df_upd = data[data['municipality'] == hotel_city].iloc[0]
    hotel_country = df_upd['iso_country']
    return hotel_country

traveler_names = []
start_dates = []
end_dates = []
no_of_rooms_list = []
no_of_nights_list = []
hotel_names = []
hotel_cities = []
hotel_countries = []
data = pd.read_excel('World Cities DB.xlsx')
for file in glob.glob('alkhoory_images/*.txt'):
    print(file)

    with open(file) as fp:
        traveler_name_final = 0
        start_date_final = 0
        end_date_final = 0
        no_of_rooms_final = 0
        no_of_nights_final = 0
        hotel_name_final = 0
        hotel_city_final = 0
        hotel_country_final = 0

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            flag = 0
            if 'room charge' in text.lower():
                flag = 1
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            if flag == 1:
                traveler_name = get_traveler_name(text)
                if traveler_name != '' and not traveler_name_final:
                    traveler_name_final = traveler_name
                # print(traveler_name)
                try:
                    start_date, end_date = get_dates(text)
                    if start_date != '' and not start_date_final:
                        start_date_final = start_date
                    if end_date != '' and not end_date_final:
                        end_date_final = end_date
                    # print(start_date)
                    # print(end_date)
                except:
                    pass
                no_of_rooms = get_number_of_rooms(text)
                if no_of_rooms != '' and not no_of_rooms_final:
                    no_of_rooms_final = no_of_rooms
                # print(no_of_rooms)
                no_of_nights = get_number_of_nights(text)
                if no_of_nights != '' and not no_of_nights_final:
                    no_of_nights_final = no_of_nights
                # print(no_of_nights)
                hotel_name = get_hotel_name(text)
                if hotel_name != '' and not hotel_name_final:
                    hotel_name_final = hotel_name
                # print(hotel_name)
                hotel_city = get_hotel_city(text)
                if hotel_city != '' and not hotel_city_final:
                    hotel_city_final = hotel_city
                # print(hotel_city)
                hotel_country = get_hotel_country(text, data)
                if hotel_country != '' and not hotel_country_final:
                    hotel_country_final = hotel_country

        traveler_names.append(traveler_name_final)
        # print(traveler_name_final)
        start_dates.append(start_date_final)
        # print(start_date_final)
        end_dates.append(end_date_final)
        # print(end_date_final)
        no_of_rooms_list.append(no_of_rooms_final)
        # print(no_of_rooms_final)
        no_of_nights_list.append(no_of_nights_final)
        # print(no_of_nights_final)
        hotel_names.append(hotel_name_final)
        # print(hotel_name_final)
        hotel_cities.append(hotel_city_final)
        # print(hotel_city_final)
        hotel_countries.append(hotel_country_final)
        # print('------------------------------------')

df = pd.DataFrame({
    'Traveler Name': traveler_names,
    'Start Date': start_dates,
    'End Date': end_dates,
    'Number of Rooms': no_of_rooms_list,
    'Number of Nights': no_of_nights_list,
    'Hotel Name': hotel_names,
    'Hotel City': hotel_cities,
    'Hotel Country': hotel_countries
})

df.to_excel('hotelstay_data.xlsx')

In [58]:
data = pd.read_excel('World Cities DB.xlsx')

In [8]:
################################ for HABLIS format ################################

from dateutil.parser import parse
import pandas as pd
import glob
import datetime

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

def hablis_traveler_name(text):
    traveler_name = ''
    for line in text:
        if 'mr ' in line.lower() or 'ms ' in line.lower():
            words = line.split()
            # print(words)
            for word in words:
                if not word.isalpha() and traveler_name == '':
                    continue
                if not word.isalpha() and traveler_name != '':
                    break
                else:
                    traveler_name += word + ' '
    return traveler_name.strip()

def hablis_start_date(text):
    start_date = ''
    flag = 0
    for i in range(len(text)):
        if 'arrival' in text[i].lower():
            words = text[i+1].split()
            # print(words)
            for word in words:
                if is_date(word) and '/' in word and ',' not in word:
                    flag = 1
                    start_date = word
                    # print('START DATE---> ', start_date)
                    break
            if flag == 1:
                break
    try:
        start_date = datetime.datetime.strptime(start_date, '%d/%m/%Y').strftime('%Y-%m-%d')
        return start_date
    except:
        return None

def hablis_end_date(text):
    end_date = ''
    flag = 0
    for i in range(len(text)):
        if 'departure' in text[i].lower():
            words = text[i+1].split()
            # print(words)
            for word in words:
                if is_date(word) and '/' in word and ',' not in word:
                    flag = 1
                    end_date = word
                    # print('END DATE---> ', end_date)
                    break
            if flag == 1:
                break
    
    try:
        end_date = datetime.datetime.strptime(end_date, '%d/%m/%Y').strftime('%Y-%m-%d')
        return end_date
    except:
        return None

def hablis_no_of_nights(text):
    no_of_nights = 0
    try:
        start_date = hablis_start_date(text)
        fyear, fmonth, fday = start_date.split('-')
        fdate = datetime.date(int(fyear), int(fmonth), int(fday))
        end_date = hablis_end_date(text)
        lyear, lmonth, lday = end_date.split('-')
        ldate = datetime.date(int(lyear), int(lmonth), int(lday))
        return (ldate-fdate).days
    except:
        pass

def hablis_no_of_rooms(text):
    no_of_rooms = 0
    for i in range(len(text)):
        if 'room' in text[i].lower():
            words = text[i+1].split()
            if words[-3].isnumeric():
                return 1

def hablis_hotel_name(text):
    hotel_name = ''
    for line in text:
        if 'hotels' in line.lower():
            hotel_name = line
            return hotel_name

def hablis_hotel_city(text, data):
    hotel_city = ''
    cities = data['city'].tolist()
    debarred = ['page', 'central']
    # pprint(cities)
    cities = [str(city).lower() for city in cities]
    for i in range(len(text)):
        words = text[i].split()
        for word in words:
            if word.lower().strip(',') in cities:
                if word.lower() not in debarred:
                    word = word[0].upper()+word[1:].lower()
                    return word.strip(',')

def hablis_hotel_country(text, data):
    hotel_city = hablis_hotel_city(text, data)
    hotel_country = ''
    if hotel_city:
        value = data[data['city'] == hotel_city.strip()].iloc[0]
        hotel_country = value['country']
        return hotel_country

traveler_names = []
start_dates = []
end_dates = []
no_of_rooms_list = []
no_of_nights_list = []
hotel_names = []
hotel_cities = []
hotel_countries = []

data = pd.read_excel('World_DB.xlsx')

for file in glob.glob('ensemble_files_hotel_stay2/*.txt'):

    with open(file) as fp:
        traveler_name_final = 0
        start_date_final = 0
        end_date_final = 0
        no_of_rooms_final = 0
        no_of_nights_final = 0
        hotel_name_final = 0
        hotel_city_final = 0
        hotel_country_final = 0

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            traveler_name = hablis_traveler_name(text)
            if traveler_name != '' and not traveler_name_final:
                traveler_name_final = traveler_name
            
            start_date = hablis_start_date(text)
            if start_date != '' and not start_date_final:
                start_date_final = start_date

            end_date = hablis_end_date(text)
            if end_date != '' and not end_date_final:
                end_date_final = end_date  

            no_of_nights = hablis_no_of_nights(text)
            if no_of_nights != '' and not no_of_nights_final:
                no_of_nights_final = no_of_nights

            no_of_rooms = hablis_no_of_rooms(text)
            if no_of_rooms != '' and not no_of_rooms_final:
                no_of_rooms_final = no_of_rooms

            hotel_name = hablis_hotel_name(text)
            if hotel_name != '' and not hotel_name_final:
                hotel_name_final = hotel_name   

            hotel_city = hablis_hotel_city(text, data)
            if hotel_city != '' and not hotel_city_final:
                hotel_city_final = hotel_city    

            hotel_country = hablis_hotel_country(text, data)
            if hotel_country != '' and not hotel_country_final:
                hotel_country_final = hotel_country

        traveler_names.append(traveler_name_final)
        # print(traveler_name_final)
        start_dates.append(start_date_final)
        # print(start_date_final)
        end_dates.append(end_date_final)
        # print(end_date_final)
        no_of_rooms_list.append(no_of_rooms_final)
        # print(no_of_rooms_final)
        no_of_nights_list.append(no_of_nights_final)
        # print(no_of_nights_final)
        hotel_names.append(hotel_name_final)
        # print(hotel_name_final)
        hotel_cities.append(hotel_city_final)
        # print(hotel_city_final)
        hotel_countries.append(hotel_country_final)
        # print('------------------------------------')

    print(file)
    print(traveler_name_final)
    print(start_date_final)
    print(end_date_final)
    print(no_of_nights_final)
    print(no_of_rooms_final)
    print(hotel_name_final)
    print(hotel_city_final)
    print(hotel_country_final)
    print('------------------------------------------------')

df = pd.DataFrame({
    'Traveler Name': traveler_names,
    'Start Date': start_dates,
    'End Date': end_dates,
    'Number of Rooms': no_of_rooms_list,
    'Number of Nights': no_of_nights_list,
    'Hotel Name': hotel_names,
    'Hotel City': hotel_cities,
    'Hotel Country': hotel_countries
})

df.to_excel('hotelstay_data2.xlsx')

ensemble_files_hotel_stay2\22089439_page0.txt
MR VENKATESH DEVADULA
None
None
None
None
HABLIS HOTELS
Chennai
IN
------------------------------------------------
ensemble_files_hotel_stay2\22089439_page1.txt
MR VENKATESH DEVADULA
2022-07-25
2022-07-29
4
None
None
Chennai
IN
------------------------------------------------
ensemble_files_hotel_stay2\8808_page0.txt
MR VENKATESH DEVADULA
2022-07-23
2022-07-24
1
1
HABLIS HOTELS
Chennai
IN
------------------------------------------------
ensemble_files_hotel_stay2\8894_page0.txt
MS SARANYA
2022-07-25
2022-07-26
1
1
HABLIS HOTELS
Chennai
IN
------------------------------------------------
ensemble_files_hotel_stay2\netlink_20726,20727,23589,21437_page0.txt
MS SARANYA
2022-02-21
2022-02-22
1
1
None
Chennai
IN
------------------------------------------------
ensemble_files_hotel_stay2\netlink_20726,20727,23589,21437_page1.txt
MR SASIKUMAR ASHOKAN
2022-02-21
2022-02-22
1
None
None
Chennai
IN
------------------------------------------------
ense

In [9]:
def split_line(line):
    words = line.split()
    newwords = [word.lower() for word in words]
    return words, newwords

# def remove_debarred(text):
#     debarred = ['netlink', 'flat', 'date', 'transaction', 'gst', 'property', 'information', 'floor', 'tower']
#     for line in text:
#         for deb in debarred:
#             if deb in line.lower():
#                 text.remove(line)
#                 break
#     return text

def return_required(words, newwords, keyword):
    ind = newwords.index(keyword)
    name = ' '.join(words[:ind])
    return name

def remove_debarred(text):
    debarred = ['netlink', 'flat', 'date', 'transaction', 'gst', 'property', 'information', 'floor', 'tower']
    # for line in text:
    #     for deb in debarred:
    #         if deb in line.lower():
    #             text.remove(line)
    #             break
    newtext = text.copy()
    for line in text:
        for deb in debarred:
            if deb in line.lower():
                # print('yeess, line---> ', line)
                newtext.remove(line)
                break
    return newtext

def clean_name(tname):
    if tname:
        words, newwords = split_line(tname)
        if 'guest:' in newwords:
            ind = newwords.index('guest:')
            tname = ' '.join(words[ind+1:])
            # return traveler_name 
        if ':' in newwords[0]:
            tname = ' '.join(words[1:])
        if 'invoice' in newwords:
            ind = newwords.index('invoice')
            tname = ' '.join(words[:ind])
        if '|' in tname:
            tname = tname.replace('|', '')
        if 'child' in newwords:
            ind = newwords.index('/')
            tname = ' '.join(words[:ind])
        for i, c in enumerate(tname):
            if c.isdigit():
                tname = tname[:i]
                break

    return tname.strip()

def get_traveler_name3(text):
    traveler_name = []
    flag = 0

    # text = remove_debarred(text)
    # pprint(text)
    text = remove_debarred(text)

    i = 0
    t_name = ''
    for line in text:
        
        if 'room no' in line.lower() or 'room number' in line.lower():
            val = 'guest name'
            if not any(val in x.lower() for x in text):
                words, newwords = split_line(line)
                if 'room' in newwords:
                    t_name = return_required(words, newwords, 'room')
                    t_name = clean_name(t_name)
                    traveler_name.append(t_name)
                    # print('ROOM')
                    break
                    # return traveler_name
        
        if 'id' in line.lower().split():
            words, newwords = split_line(line)
            t_name = return_required(words, newwords, 'id')
            t_name = clean_name(t_name)
            traveler_name.append(t_name)
            # print('ID')
            break
            # return traveler_name
        
        if 'folio' in line.lower():
            words, newwords = split_line(line)
            t_name = return_required(words, newwords, 'folio')
            t_name = clean_name(t_name)
            traveler_name.append(t_name)
            break
            # return traveler_name
        
        if 'guest name' in line.lower():
            if 'room charges' not in line.lower():
                words, newwords = split_line(line)
                if 'departure' in newwords:
                    t_name = return_required(words, newwords, 'departure')
                    t_name = clean_name(t_name)
                    traveler_name.append(t_name)
                    # print('DEPARTURE')
                    break
                    # return traveler_name
                elif 'child' in newwords:
                    t_name = ' '.join(words[2:])
                    t_name = clean_name(t_name)
                    traveler_name.append(t_name)
                    break
                    # return traveler_name
                # elif 'mr' in text[i+1].lower():
                #     for j in range(i+1, len(text)):
                        
                else:
                    # print(words)
                    t_name = ' '.join(words[3:])
                    t_name = clean_name(t_name)
                    traveler_name.append(t_name)
                    break
                    # return traveler_name
        
        val = 'guest name'
        if not any(val in x.lower() for x in text):
            if 'mr' in line.lower() and 'omr' not in line.lower():
            #     words, newwords = split_line(line)
                if 'room' not in line.lower():
                    # print('ROOM 2')
                    # print('LINE--->', line)
                    if 'invoice' in line.lower():
                        words, newwords = split_line(line)
                        t_name = return_required(words, newwords, 'invoice')
                        t_name = clean_name(t_name)
                        traveler_name.append(t_name)
                        break
                        # return traveler_name
                    t_name = line
                    t_name = clean_name(t_name)
                    traveler_name.append(t_name)
                    break
                    # return traveler_name
                else:
                    # print('ELSE ROOM 2')
                    words, newwords = split_line(line)
                    if 'room' in newwords:
                        t_name = return_required(words, newwords, 'room')
                        t_name = clean_name(t_name)
                        traveler_name.append(t_name)
                        break
                        # return traveler_name
        
        if 'name:' in line.lower():
            words, newwords = split_line(line)
            ind = newwords.index('name:')
            t_name = ' '.join(words[ind+1:])
            t_name = clean_name(t_name)
            traveler_name.append(t_name)
            break
            # return traveler_name
        
        if 'received from' in line.lower():
            words, newwords = split_line(line)
            ind = newwords.index('transfer:')
            t_name = ' '.join(words[2:ind])
            t_name = clean_name(t_name)
            traveler_name.append(t_name)
            break

        if 'to room charges' in line.lower():
            flag = 1
            for j in range(i+1, len(text)):
                if 'total:' in text[j].lower():
                    break
                words, newwords = split_line(text[j])
                ind = newwords.index('single')
                t_name = ' '.join(words[ind+1:])
                t_name = clean_name(t_name)
                traveler_name.append(t_name)

        i += 1
    # print(traveler_name)
    # if flag == 0:
    #     # print('YEESSSS')
    #     t_name = clean_name(t_name)
    #     traveler_name.append(t_name)

    # print('TRAVELER NAME---> ', traveler_name)
    return traveler_name

In [1]:
import json

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

# def get_traveler_name3(text):
#     traveler_name = ''
#     debarred = ['netlink', 'flat', 'date', 'transaction', 'gst', 'property', 'information', 'floor', 'tower']
#     flag = 0
#     for line in text:

#         if 'room no' in line.lower() or 'room number' in line.lower():
#             val = 'guest name'
#             if not any(val in x.lower() for x in text):
#                 words = line.split()
#                 newwords = [word.lower() for word in words]
#                 ind = newwords.index('room')
#                 traveler_name = ' '.join(words[:ind])
#                 for deb in debarred:
#                     if deb in traveler_name.lower():
#                         traveler_name = ''
#                         flag = 1
#                         break
#                 if flag == 0:
#                     return traveler_name
#         flag = 0
#         if 'id' in line.lower().split():
#             words = line.split()
#             newwords = [word.lower() for word in words]
#             for newword in newwords:
#                 if newword in debarred:
#                     flag = 1
#                     break
#             if flag == 0:
#                 ind = newwords.index('id')
#                 traveler_name = ' '.join(words[:ind])
#                 return traveler_name
#         flag = 0
#         if 'folio' in line.lower():
#             words = line.split()
#             newwords = [word.lower() for word in words]
#             for newword in newwords:
#                 if newword in debarred:
#                     flag = 1
#                     break
#             if flag == 0:
#                 ind = newwords.index('folio')
#                 traveler_name = ' '.join(words[:ind])
#                 return traveler_name
#         flag = 0
#         if 'guest name' in line.lower():
#             words = line.split()
#             newwords = [word.lower() for word in words]
#             if 'departure' in newwords:
#                 ind = newwords.index('departure')
#                 traveler_name = ' '.join(words[3:ind])
#                 return traveler_name
#             else:
#                 traveler_name = ' '.join(words[3:])
#                 return traveler_name
#         flag = 0
#         val = 'guest name'
#         if not any(val in x.lower() for x in text):
#             if 'mr' in line.lower() and 'omr' not in line.lower():
#                 words = line.split()
#                 newwords = [word.lower() for word in words]
#                 for newword in newwords:
#                     if newword in debarred:
#                         flag = 1
#                         break
#                 if flag == 0:
#                     if 'room' not in line.lower():
#                         traveler_name = line
#                         return traveler_name
#                     else:
#                         words = line.split()
#                         newwords = [word.lower() for word in words]
#                         ind = newwords.index('room')
#                         traveler_name = ' '.join(words[:ind])
#                         return traveler_name

def get_start_date3(text):
    start_date = []
    s_date = ''
    formats = ['%d/%m/%Y', '%d-%b-%y', '%d-%m-%y', '%d/%m/%y', '%d-%b-%Y']
    i = 0
    for line in text:
        if 'arrival departure night' in line.lower():
            s_date = text[i+1].split()[0].strip('.')
            for format in formats:
                try:
                    s_date = datetime.datetime.strptime(s_date, format).strftime('%Y-%m-%d')
                    start_date.append(s_date)
                except:
                    pass
            return start_date
        if 'arrival' in line.lower() or 'period from' in line.lower() or 'ch. in date' in line.lower():
            words = line.split()
            for word in words:
                if is_date(word.strip(':')):
                    s_date = word.strip(':')
                    for format in formats:
                        try:
                            s_date = datetime.datetime.strptime(s_date, format).strftime('%Y-%m-%d')
                            start_date.append(s_date)
                        except:
                            pass
                    return start_date
        if 'to room charges' in line.lower():
            for j in range(i+1, len(text)):
                if 'total:' in text[j].lower():
                    break
                s_date = text[j].split()[0]
                for format in formats:
                    try:
                        s_date = datetime.datetime.strptime(s_date, format).strftime('%Y-%m-%d')
                        start_date.append(s_date)
                    except:
                        pass
            return start_date
        i += 1

def get_end_date3(text):
    end_date = []
    e_date = ''
    formats = ['%d/%m/%Y', '%d-%b-%y', '%d-%m-%y', '%d/%m/%y', '%d-%b-%Y']
    i = 0
    for line in text:
        if 'arrival departure night' in line.lower():
            e_date = text[i+1].split()[1].strip('.')
            for format in formats:
                try:
                    e_date = datetime.datetime.strptime(e_date, format).strftime('%Y-%m-%d')
                    end_date.append(e_date)
                except:
                    pass
            return end_date
        if 'departure' in line.lower():
            words = line.split()
            newwords = line.lower().split()
            ind = newwords.index('departure')
            words = words[ind:]
            for word in words:
                if is_date(word.strip(':')):
                    e_date = word.strip(':')
                    for format in formats:
                        try:
                            e_date = datetime.datetime.strptime(e_date, format).strftime('%Y-%m-%d')
                            end_date.append(e_date)
                        except:
                            pass
                    return end_date
        if 'period from' in line.lower():
            newwords = line.lower().split()
            ind = newwords.index('to')
            e_date = newwords[ind+1]
            for format in formats:
                try:
                    e_date = datetime.datetime.strptime(e_date, format).strftime('%Y-%m-%d')
                    end_date.append(e_date)
                except:
                    pass
            return end_date
        if 'ch. out date' in line.lower():
            words = line.split()
            for word in words:
                if is_date(word) and '/' in word:
                    e_date = word
                    for format in formats:
                        try:
                            e_date = datetime.datetime.strptime(e_date, format).strftime('%Y-%m-%d')
                            end_date.append(e_date)
                        except:
                            pass
                    return end_date
        if 'to room charges' in line.lower():
            for j in range(i+1, len(text)):
                if 'total:' in text[j].lower():
                    break
                e_date = text[j].split()[1]
                if 'O' in e_date:
                    e_date = e_date.replace('O', '0')
                for format in formats:
                    try:
                        e_date = datetime.datetime.strptime(e_date, format).strftime('%Y-%m-%d')
                        end_date.append(e_date)
                    except:
                        pass
            return end_date
        i += 1

def get_no_of_rooms3(text):
    no_of_rooms = []
    rooms = 0
    for line in text:
        if 'room no' in line.lower() or 'room number' in line.lower():
            words = line.lower().split()
            if 'room' in words:
                ind = words.index('room')
                words = words[ind:]
                for word in words:
                    if word.isnumeric():
                        rooms += 1
                no_of_rooms.append(rooms)
                return no_of_rooms

def get_no_of_nights3(text):
    no_of_nights = []
    nights = 0
    try:
        start_date = get_start_date3(text)
        end_date = get_end_date3(text)
        for i in range(len(start_date)):
            fyear, fmonth, fday = start_date[i].split('-')
            fdate = datetime.date(int(fyear), int(fmonth), int(fday))
            lyear, lmonth, lday = end_date[i].split('-')
            ldate = datetime.date(int(lyear), int(lmonth), int(lday))
            nights = (ldate-fdate).days
            no_of_nights.append(nights)
        return no_of_nights
    except:
        pass

def get_hotel_name3(text):
    hotel_name = []
    h_name = ''
    debarred = ['seat', 'a', 'expertise', 'un', 'room', 'siar']
    name_found = 0
    for line in text:
        if 'hilton garden inn' in line.lower():
            name_found = 1
            h_name = 'Hilton Garden Inn'
            break
        if 'doubletree by hilton' in line.lower():
            name_found = 1
            h_name = 'DOUBLETREE BY HILTON'
            break
        if 'park hyatt' in line.lower():
            name_found = 1
            h_name = 'PARK HYATT'
            break
        if 'courtyard by marriott' in line.lower():
            name_found = 1
            h_name = 'Courtyard by Marriott'
            break
        if 'park hotels' in line.lower():
            name_found = 1
            h_name = 'THE PARK Hotels'
            break

    i = 0
    if name_found == 0:
        for line in text:
            if i == 0 or i == 1:
                flag = 0
                if text[i].lower() not in debarred:
                    if 'vat' in text[i].lower():
                        words = text[i].split()
                        newwords = text[i].lower().split()
                        ind = newwords.index('vat')
                        flag = 1
                        h_name = ' '.join(words[:ind])
                    if not flag:
                        h_name = text[i]
                        break
            i += 1

    hotel_name.append(h_name)
    return hotel_name

def replace(string, ch):
    string = string.replace(ch, '')
    return string

def get_hotel_city3(text):
    hotel_city = []
    h_city = ''
    debarred = ['page', 'po', 'ye', 'lock', 'sola', 'au', 'bo', 'ati', 'sur', 'osh']
    for line in text:
        for city in cities:
            if city in line.lower():
                if city not in debarred:
                    h_city = city
                    h_city = h_city.split()
                    for i in range(len(h_city)):
                        h_city[i] = h_city[i][0].upper()+h_city[i][1:].lower()
                    h_city = ' '.join(h_city)
                    hotel_city.append(h_city)
                    return hotel_city

def get_hotel_country3(text, data):
    hotel_country = []
    h_country = ''
    hotel_city = get_hotel_city3(text)
    
    if hotel_city:
        for city in hotel_city:
            if city:
                value = data[data['city'] == city.strip()].iloc[0]
                h_country = value['country']
                hotel_country.append(h_country)
                return hotel_country

f = open('cities.json')
cities = json.load(f)
cities = {k.lower(): v for k, v in cities.items()}
f.close()
data = pd.read_excel('World_DB.xlsx')

traveler_names = []
start_dates = []
end_dates = []
no_of_rooms_list = []
no_of_nights_list = []
hotel_names = []
hotel_cities = []
hotel_countries = []

for file in glob.glob('ensemble_files_hotel_stay4/*.txt'):
    print(file)

    with open(file) as fp:
        traveler_name_final = []
        start_date_final = []
        end_date_final = []
        no_of_rooms_final = []
        no_of_nights_final = []
        hotel_name_final = []
        hotel_city_final = []
        hotel_country_final = []

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            # pprint(text)
            filename = os.path.split(file)[-1].split('.txt')[0]

            traveler_name = get_traveler_name3(text)
            if traveler_name and not traveler_name_final:
                traveler_name_final = traveler_name
            start_date = get_start_date3(text)
            if start_date and not start_date_final:
                start_date_final = start_date
            end_date = get_end_date3(text)
            if end_date and not end_date_final:
                end_date_final = end_date
            no_of_nights = get_no_of_nights3(text)
            if no_of_nights != '' and not no_of_nights_final:
                no_of_nights_final = no_of_nights
            hotel_name = get_hotel_name3(text)
            if hotel_name != '' and not hotel_name_final:
                hotel_name_final = hotel_name
            hotel_city = get_hotel_city3(text)
            if hotel_city != '' and not hotel_city_final:
                hotel_city_final = hotel_city
            hotel_country = get_hotel_country3(text, data)
            if hotel_country != '' and not hotel_country_final:
                hotel_country_final = hotel_country
            no_of_rooms = get_no_of_rooms3(text)
            if no_of_rooms != '' and not no_of_rooms_final:
                no_of_rooms_final = no_of_rooms

            traveler_name_new = ' '.join([str(elem) for elem in traveler_name_final])
            traveler_names.append(traveler_name_new)
            # print(traveler_name_final)

            start_dates_new = ' '.join([str(elem) for elem in start_date_final])
            start_dates.append(start_dates_new)
            
            # print(start_date_final)
            end_dates_new = ' '.join([str(elem) for elem in end_date_final])
            end_dates.append(end_dates_new)
            
            # print(end_date_final)
            no_of_rooms_list_new = ' '.join([str(elem) for elem in no_of_rooms_final or []])
            no_of_rooms_list.append(no_of_rooms_list_new)
            
            # print(no_of_rooms_final)
            no_of_nights_list_new = ' '.join([str(elem) for elem in no_of_nights_final or []])
            no_of_nights_list.append(no_of_nights_list_new)
            
            # print(no_of_nights_final)
            hotel_names_new = ' '.join([str(elem) for elem in hotel_name_final])
            hotel_names.append(hotel_names_new)
            
            # print(hotel_name_final)
            hotel_cities_new = ' '.join([str(elem) for elem in hotel_city_final or []])
            hotel_cities.append(hotel_cities_new)
            
            # print(hotel_city_final)
            hotel_countries_new = ' '.join([str(elem) for elem in hotel_country_final or []])
            hotel_countries.append(hotel_countries_new)
            
            # print('------------------------------------')

    # print(traveler_name_final)
    # print(start_date_final)
    # print(end_date_final)
    # print(no_of_rooms_final)
    # print(no_of_nights_final)
    # print(hotel_name_final)
    # print(hotel_city_final)
    # print(hotel_country_final)
    # print('----------------------------------------')

# print(traveler_names)
# traveler_names_new = ' '.join([str(elem) for elem in traveler_names])
# print(traveler_names_new)
# start_dates_new = ' '.join([str(elem) for elem in start_dates])
# end_dates_new = ' '.join([str(elem) for elem in end_dates])
# no_of_rooms_list_new = ' '.join([str(elem) for elem in no_of_rooms_list])
# no_of_nights_list_new = ' '.join([str(elem) for elem in no_of_nights_list])
# hotel_names_new = ' '.join([str(elem) for elem in hotel_names])
# hotel_cities_new = ' '.join([str(elem) for elem in hotel_cities])
# hotel_countries_new = ' '.join([str(elem) for elem in hotel_countries])

# df = pd.DataFrame({
#     'Traveler Name': traveler_names_new,
#     'Start Date': start_dates_new,
#     'End Date': end_dates_new,
#     'Number of Rooms': no_of_rooms_list_new,
#     'Number of Nights': no_of_nights_list_new,
#     'Hotel Name': hotel_names_new,
#     'Hotel City': hotel_cities_new,
#     'Hotel Country': hotel_countries_new
# })

df = pd.DataFrame({
    'Traveler Name': traveler_names,
    'Start Date': start_dates,
    'End Date': end_dates,
    'Number of Rooms': no_of_rooms_list,
    'Number of Nights': no_of_nights_list,
    'Hotel Name': hotel_names,
    'Hotel City': hotel_cities,
    'Hotel Country': hotel_countries
})

df.to_excel('hotelstay_data4.xlsx')

NameError: name 'pd' is not defined

In [ ]:
import json

cities = data['city'].to_json()
cities = json.loads(cities)
newcities = {value: key for key, value in cities.items()}
out_file = open("cities.json", "w")
json.dump(newcities, out_file, indent = 6)
out_file.close()
pprint(newcities)

In [249]:
print(len(glob.glob('hotel_stay_images3/*')))

37
